<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/brain_tumor_hf_ds_protoype_analysis_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain Tumor Image Dataset - Prototype Analysis

- HF DataSet Loader



## Set up and Initial Checks

### Import Necessary Libraries and Define Constants

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -qU pyarrow==14.0.1 requests==2.31.0

In [3]:
!pip install -qU datasets==2.11.0

In [4]:
import os
import json
import pandas as pd
from datasets import Dataset, Features, ClassLabel, Value, Sequence, Image
from tqdm.auto import tqdm
import cv2

In [5]:
HF_DATASET_NAME = 'brain-tumor-image-dataset-semantic-segmentation'
SOURCE_JSON = "/content/drive/MyDrive/kaggle/datasets/brain-tumor-image-dataset-semantic-segmentation/train/_annotations.coco.json"
SOURCE_IMAGE_DIR = "/content/drive/MyDrive/kaggle/datasets/brain-tumor-image-dataset-semantic-segmentation/train"


### Initial Verification

In [6]:
features = Features({
    'file_name': Value(dtype='string'),
    'image': Image(),
    'id': Value(dtype='int64'),
    'category_id': ClassLabel(names=['Tumor', '0', '1']),
    'bbox': Sequence(feature=Value(dtype='float32'), length=4),
    'segmentation': Sequence(Sequence(Value(dtype='float32'))),
    'area': Value(dtype='float32'),
    'iscrowd': Value(dtype='int64'),
    'height': Value(dtype='int64'),
    'width': Value(dtype='int64'),
    'date_captured': Value(dtype='string'),
    'license': Value(dtype='int64')
})

In [7]:
def verify_source_data():
    with open(SOURCE_JSON, 'r') as f:
        data = json.load(f)

    print("Categories:")
    for category in data['categories']:
        print(f"ID: {category['id']}, Name: {category['name']}, Supercategory: {category['supercategory']}")

    category_counts = pd.DataFrame(data['annotations'])['category_id'].value_counts().sort_index()
    print("\nCategory distribution in annotations:")
    print(category_counts)

    # Check for images with multiple bounding boxes
    image_bbox_counts = pd.DataFrame(data['annotations'])['image_id'].value_counts()
    print(f"\nImages with multiple bounding boxes: {(image_bbox_counts > 1).sum()}")
    print(f"Max bounding boxes in an image: {image_bbox_counts.max()}")

In [8]:
def load_data_to_df():
    with open(SOURCE_JSON, 'r') as f:
        data = json.load(f)

    images = pd.DataFrame(data['images'])
    annotations = pd.DataFrame(data['annotations'])

    df = pd.merge(images, annotations, left_on='id', right_on='image_id', suffixes=('', '_ann'))

    # Drop duplicate columns
    df = df.drop(columns=['id_ann', 'image_id'])

    # Add the full image path
    df['image'] = df['file_name'].apply(lambda x: os.path.join(SOURCE_IMAGE_DIR, x))

    # Ensure all required columns are present
    for column in features.keys():
        if column not in df.columns and column != 'image':
            df[column] = None

    return df

In [9]:
def create_hf_dataset(df, hf_dataset_name):
    # Convert 'image' column to image data
    def load_image(image_path):
        img = cv2.imread(image_path)
        if img is not None:
            return cv2.imencode('.jpg', img)[1].tobytes()
        return None

    tqdm.pandas(desc="Loading images")
    df['image'] = df['image'].progress_apply(load_image)

    # Ensure datatypes match the features
    df['bbox'] = df['bbox'].apply(lambda x: [float(i) for i in x])
    df['segmentation'] = df['segmentation'].apply(lambda x: [[float(i) for i in poly] for poly in x])
    df['area'] = df['area'].astype('float32')

    dataset = Dataset.from_pandas(df, features=features)
    print(f"Dataset created successfully with {len(dataset)} examples.")
    return dataset

In [ ]:
def verify_dataset(dataset):
    print(f"Dataset contains {len(dataset)} examples.")
    print("Sample of the first example:")
    print(dataset[0])

    # Check if all required fields are present
    required_fields = ['file_name', 'image', 'id', 'category_id', 'bbox', 'segmentation', 'area']
    for field in required_fields:
        if field not in dataset[0]:
            print(f"Warning: '{field}' is missing from the dataset.")

    # Verify image data
    if isinstance(dataset[0]['image'], bytes):
        print("Image data is stored as bytes.")
    else:
        print("Warning: Image data is not stored as bytes.")

    # Verify segmentation data
    if isinstance(dataset[0]['segmentation'], list) and isinstance(dataset[0]['segmentation'][0], list):
        print("Segmentation data is stored as a list of lists.")
    else:
        print("Warning: Segmentation data is not stored as a list of lists.")

    print("Dataset verification complete.")

In [10]:
# Load and prepare the data
df = load_data_to_df()

# Create the dataset
dataset = create_hf_dataset(df, HF_DATASET_NAME)

# Verify the dataset
verify_dataset(dataset)

# If you want to upload to the Hub (uncomment when ready)
# dataset.push_to_hub(HF_DATASET_NAME)

Categories:
ID: 0, Name: Tumor, Supercategory: none
ID: 1, Name: 0, Supercategory: Tumor
ID: 2, Name: 1, Supercategory: Tumor

Category distribution in annotations:
category_id
1    771
2    731
Name: count, dtype: int64

Images with multiple bounding boxes: 1
Max bounding boxes in an image: 2


Creating dataset:   0%|          | 0/3 [00:00<?, ?it/s]

Loading images:   0%|          | 0/1502 [00:00<?, ?it/s]

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset brain-tumor-image-dataset-semantic-segmentation created successfully.


In [11]:
import json
from collections import Counter

with open(SOURCE_JSON, 'r') as f:
    coco_data = json.load(f)

category_ids = [ann['category_id'] for ann in coco_data['annotations']]
id_counts = Counter(category_ids)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

print("Category ID counts:", id_counts)
print("Category mappings:", categories)

# Automated verification
expected_categories = {0: 'Tumor', 1: '0', 2: '1'}
assert categories == expected_categories, f"Category mismatch. Expected {expected_categories}, got {categories}"
assert set(id_counts.keys()) == {1, 2}, f"Unexpected category IDs found: {set(id_counts.keys())}"

Category ID counts: Counter({1: 771, 2: 731})
Category mappings: {0: 'Tumor', 1: '0', 2: '1'}


In [13]:
def is_rectangle_segmentation(bbox, segmentation):
    x, y, w, h = bbox
    expected = [[x, y, x+w, y, x+w, y+h, x, y+h]]
    if segmentation != expected:
        print(f"Non-rectangular segmentation found:")
        print(f"Bbox: {bbox}")
        print(f"Segmentation: {segmentation}")
        print(f"Expected: {expected}")
        print("---")
    return segmentation == expected

sample_annotations = coco_data['annotations'][:100]
rectangle_count = sum(is_rectangle_segmentation(ann['bbox'], ann['segmentation'])
                      for ann in sample_annotations)

rectangle_percentage = (rectangle_count / len(sample_annotations)) * 100
print(f"{rectangle_percentage:.2f}% of sampled annotations have rectangular segmentations")

# Instead of asserting, let's just print a warning
if rectangle_percentage < 100:
    print(f"Warning: Only {rectangle_percentage:.2f}% of segmentations are rectangular")

Non-rectangular segmentation found:
Bbox: [145, 239, 168.75, 162.5]
Segmentation: [[313.75, 238.75, 145, 238.75, 145, 401.25, 313.75, 401.25, 313.75, 238.75]]
Expected: [[145, 239, 313.75, 239, 313.75, 401.5, 145, 401.5]]
---
Non-rectangular segmentation found:
Bbox: [194, 176, 148.75, 233.75]
Segmentation: [[342.5, 176.25, 193.75, 176.25, 193.75, 410, 342.5, 410, 342.5, 176.25]]
Expected: [[194, 176, 342.75, 176, 342.75, 409.75, 194, 409.75]]
---
Non-rectangular segmentation found:
Bbox: [133, 173, 162.5, 185]
Segmentation: [[295, 172.5, 132.5, 172.5, 132.5, 357.5, 295, 357.5, 295, 172.5]]
Expected: [[133, 173, 295.5, 173, 295.5, 358, 133, 358]]
---
Non-rectangular segmentation found:
Bbox: [245, 358, 138.75, 166.25]
Segmentation: [[383.75, 357.5, 245, 357.5, 245, 523.75, 383.75, 523.75, 383.75, 357.5]]
Expected: [[245, 358, 383.75, 358, 383.75, 524.25, 245, 524.25]]
---
Non-rectangular segmentation found:
Bbox: [80, 189, 112.5, 132.5]
Segmentation: [[192.5, 188.75, 80, 188.75, 80, 32

In [14]:
def analyze_segmentations(annotations):
    segmentation_types = {}
    for ann in annotations:
        seg_type = (len(ann['segmentation']), len(ann['segmentation'][0]) if ann['segmentation'] else 0)
        segmentation_types[seg_type] = segmentation_types.get(seg_type, 0) + 1

    print("Segmentation structure analysis:")
    for (outer_len, inner_len), count in segmentation_types.items():
        print(f"Outer length: {outer_len}, Inner length: {inner_len}, Count: {count}")

analyze_segmentations(coco_data['annotations'][:100])

Segmentation structure analysis:
Outer length: 1, Inner length: 10, Count: 100


In [16]:
import numpy as np

def is_approximately_rectangular(bbox, segmentation, tolerance=1.0):
    x, y, w, h = bbox
    expected = np.array([[x, y], [x+w, y], [x+w, y+h], [x, y+h], [x, y]])
    actual = np.array(segmentation[0]).reshape(-1, 2)
    return np.allclose(actual, expected, atol=tolerance)

sample_annotations = coco_data['annotations'][:100]
rectangle_count = sum(is_approximately_rectangular(ann['bbox'], ann['segmentation'])
                      for ann in sample_annotations)

rectangle_percentage = (rectangle_count / len(sample_annotations)) * 100
print(f"{rectangle_percentage:.2f}% of sampled annotations are approximately rectangular")

0.00% of sampled annotations are approximately rectangular


In [15]:
def is_segmentation_inside_bbox(bbox, segmentation):
    x, y, w, h = bbox
    for polygon in segmentation:
        for i in range(0, len(polygon), 2):
            if polygon[i] < x or polygon[i] > x + w or polygon[i+1] < y or polygon[i+1] > y + h:
                return False
    return True

inside_bbox_count = sum(is_segmentation_inside_bbox(ann['bbox'], ann['segmentation'])
                        for ann in coco_data['annotations'][:100])
inside_bbox_percentage = (inside_bbox_count / 100) * 100
print(f"{inside_bbox_percentage:.2f}% of segmentations are inside their bounding boxes")

2.00% of segmentations are inside their bounding boxes


In [17]:
def is_segmentation_near_bbox(bbox, segmentation, tolerance=1.0):
    x, y, w, h = bbox
    for polygon in segmentation:
        for i in range(0, len(polygon), 2):
            if (polygon[i] < x - tolerance or
                polygon[i] > x + w + tolerance or
                polygon[i+1] < y - tolerance or
                polygon[i+1] > y + h + tolerance):
                return False
    return True

near_bbox_count = sum(is_segmentation_near_bbox(ann['bbox'], ann['segmentation'])
                      for ann in coco_data['annotations'][:100])
near_bbox_percentage = (near_bbox_count / 100) * 100
print(f"{near_bbox_percentage:.2f}% of segmentations are near their bounding boxes")

100.00% of segmentations are near their bounding boxes


In [18]:
def is_valid_polygon(segmentation):
    # Check if it's a list of lists
    if not isinstance(segmentation, list) or not all(isinstance(poly, list) for poly in segmentation):
        return False

    # Check if each polygon has at least 6 coordinates (3 points)
    if not all(len(poly) >= 6 and len(poly) % 2 == 0 for poly in segmentation):
        return False

    return True

sample_annotations = coco_data['annotations'][:100]
valid_count = sum(is_valid_polygon(ann['segmentation']) for ann in sample_annotations)
valid_percentage = (valid_count / len(sample_annotations)) * 100
print(f"{valid_percentage:.2f}% of sampled annotations have valid polygon segmentations")

100.00% of sampled annotations have valid polygon segmentations


In [19]:
import pandas as pd
from datasets import Dataset

# Create small subset
subset_images = coco_data['images'][:10]
subset_annotations = [ann for ann in coco_data['annotations']
                      if ann['image_id'] in [img['id'] for img in subset_images]]

# Create DataFrame
df_subset = pd.DataFrame({
    'file_name': [img['file_name'] for img in subset_images],
    'image_id': [img['id'] for img in subset_images],
    'category_id': [ann['category_id'] for ann in subset_annotations],
    'bbox': [ann['bbox'] for ann in subset_annotations],
    'segmentation': [ann['segmentation'] for ann in subset_annotations]
})

# Convert to Parquet
df_subset.to_parquet('test_subset.parquet')

# Load Parquet file
loaded_df = pd.read_parquet('test_subset.parquet')

# Verify data
assert len(loaded_df) == len(df_subset), "Row count mismatch"
for column in df_subset.columns:
    if column in ['bbox', 'segmentation']:
        assert all(loaded_df[column].apply(str) == df_subset[column].apply(str)), f"Mismatch in column {column}"
    else:
        assert (loaded_df[column] == df_subset[column]).all(), f"Mismatch in column {column}"

print("Parquet conversion and loading test passed successfully")

AssertionError: Mismatch in column bbox

In [20]:
from datasets import Dataset
import cv2
import matplotlib.pyplot as plt
import os

def load_image_on_demand(example):
    image_path = os.path.join(SOURCE_IMAGE_DIR, example['file_name'])
    example['image'] = cv2.imread(image_path)
    return example

# Create dataset
dataset = Dataset.from_parquet('test_subset.parquet')

# Set transform for on-demand loading
dataset.set_transform(load_image_on_demand)

# Test accessing items
for i in range(min(3, len(dataset))):
    item = dataset[i]
    assert 'image' in item, f"Image not loaded for item {i}"
    assert item['image'] is not None, f"Image is None for item {i}"
    print(f"Successfully loaded image for item {i}")

    # Visualize the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(item['image'], cv2.COLOR_BGR2RGB))
    plt.title(f"Image {i}: {item['file_name']}")
    plt.axis('off')
    plt.show()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-5e88784f05f9b898/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'